In [1]:
class Opion():
    
    def __init__(self):
            
        self.dataroot= r'C:\Users\ilasy\Downloads\EndoCV2020-Endoscopy-Disease-Detection-Segmentation-subChallenge_data\EndoCV2020-Endoscopy-Disease-Detection-Segmentation-subChallenge_data\originalImages' #image dataroot
        self.maskroot= r'C:\Users\ilasy\Downloads\irregular_mask\irregular_mask\disocclusion_img_mask'#mask dataroot
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=256 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# seconde stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='CSA_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'.\checkpoints' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='center'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=''
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=True

    

In [2]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
opt = Opion()
transform_mask = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_train = Data_load(opt.dataroot, opt.maskroot, transform, transform_mask)
iterator_train = (data.DataLoader(dataset_train, batch_size=opt.batchSize,shuffle=True))
print(len(dataset_train))
model = create_model(opt)
total_steps = 0


386
csa_net


C:\Users\ilasy\anaconda3\envs\llama2_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ilasy\anaconda3\envs\llama2_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize network with normal


C:\Users\ilasy\Downloads\CSA-inpainting-master\CSA-inpainting-master Final\models\networks.py:53: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 0.0, gain)
C:\Users\ilasy\Downloads\CSA-inpainting-master\CSA-inpainting-master Final\models\networks.py:63: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


initialize network with normal
initialize network with normal
initialize network with normal
---------- Networks initialized -------------
UnetGeneratorCSA(
  (model): UnetSkipConnectionBlock_3(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): UnetSkipConnectionBlock_3(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3), dilation=(2, 2))
          (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (3): LeakyReLU(negative_slope=0.2, inplace=True)
          (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (6): UnetSkipConnectionBlock_3(
            (model): Sequential(
              (0): LeakyReLU(negative_slope=0.2, inplace=True)
  

In [3]:
print(len(dataset_train))

386


In [ ]:
save_dir = r"C:\Users\ilasy\Downloads\CSA-inpainting-master\CSA-inpainting-master Final\images_1"
iter_start_time = time.time()
for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):
    
    epoch_start_time = time.time()
    epoch_iter = 0

#     image, mask, gt = [x.cuda() for x in next(iterator_train)]
    for image, mask in (iterator_train):
        image=image.cuda()
        mask=mask.cuda()
        mask=mask[0][0]
        mask=torch.unsqueeze(mask,0)
        mask=torch.unsqueeze(mask,1)
        mask=mask.byte()

        total_steps += opt.batchSize
        epoch_iter += opt.batchSize
        model.set_input(image,mask) # it not only sets the input data with mask, but also sets the latent mask.
        model.set_gt_latent()
        model.optimize_parameters()

        if total_steps %opt.display_freq== 0:
            real_A,real_B,fake_B=model.get_current_visuals()
            #real_A=input, real_B=ground truth fake_b=output
            pic = (torch.cat([real_A, real_B,fake_B], dim=0) + 1) / 2.0
            torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
            save_dir, epoch, total_steps + 1, len(dataset_train)), nrow=2)
        if total_steps %1== 0:
            errors = model.get_current_errors()
            t = (time.time() - iter_start_time) / opt.batchSize
            print(errors)

    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' %
                (epoch, total_steps))
        model.save(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' %
            (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    model.update_learning_rate()


C:\Users\ilasy\Downloads\CSA-inpainting-master\CSA-inpainting-master Final\models\CSA.py:124: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cuda\Indexing.cu:1441.)
  self.input_A.narrow(1,0,1).masked_fill_(self.mask_global, 2*123.0/255.0 - 1.0)
C:\Users\ilasy\Downloads\CSA-inpainting-master\CSA-inpainting-master Final\models\CSA.py:125: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cuda\Indexing.cu:1441.)
  self.input_A.narrow(1,1,1).masked_fill_(self.mask_global, 2*104.0/255.0 - 1.0)
C:\Users\ilasy\Downloads\CSA-inpainting-master\CSA-inpainting-master Final\models\CSA.py

OrderedDict([('G_GAN', 14.731237411499023), ('G_L1', 103.19992065429688), ('D', 2.0972938537597656), ('F', 2.2787933349609375)])
OrderedDict([('G_GAN', 14.331802368164062), ('G_L1', 88.30522155761719), ('D', 4.087989807128906), ('F', 1.9411470890045166)])
OrderedDict([('G_GAN', 7.909736156463623), ('G_L1', 71.3814468383789), ('D', 9.084613800048828), ('F', 1.78863525390625)])
OrderedDict([('G_GAN', 5.555576324462891), ('G_L1', 87.5174560546875), ('D', 3.359513282775879), ('F', 1.5603973865509033)])
OrderedDict([('G_GAN', 6.619771957397461), ('G_L1', 58.73507308959961), ('D', 1.7149994373321533), ('F', 1.2643375396728516)])
OrderedDict([('G_GAN', 6.841765403747559), ('G_L1', 50.14315414428711), ('D', 1.7481882572174072), ('F', 1.2068657875061035)])
OrderedDict([('G_GAN', 9.270610809326172), ('G_L1', 100.2930908203125), ('D', 2.373587131500244), ('F', 1.1846177577972412)])
OrderedDict([('G_GAN', 8.557415962219238), ('G_L1', 71.27668762207031), ('D', 2.328005075454712), ('F', 0.9946606159

OrderedDict([('G_GAN', 5.75856351852417), ('G_L1', 26.135305404663086), ('D', 0.9747016429901123), ('F', 0.12742425501346588)])
OrderedDict([('G_GAN', 6.849241733551025), ('G_L1', 45.07069396972656), ('D', 0.48647141456604004), ('F', 0.13591445982456207)])
OrderedDict([('G_GAN', 6.444026947021484), ('G_L1', 23.634878158569336), ('D', 0.5710334777832031), ('F', 0.1322488933801651)])
OrderedDict([('G_GAN', 6.49678897857666), ('G_L1', 35.49968719482422), ('D', 0.7907090187072754), ('F', 0.11996237933635712)])
OrderedDict([('G_GAN', 6.839007377624512), ('G_L1', 41.692230224609375), ('D', 0.5790313482284546), ('F', 0.1093825176358223)])
OrderedDict([('G_GAN', 5.841983318328857), ('G_L1', 41.40321731567383), ('D', 1.1375715732574463), ('F', 0.09489225596189499)])
OrderedDict([('G_GAN', 5.303108215332031), ('G_L1', 39.462974548339844), ('D', 1.1540255546569824), ('F', 0.13669410347938538)])
OrderedDict([('G_GAN', 6.229742527008057), ('G_L1', 31.71529197692871), ('D', 0.5864230394363403), ('F'

OrderedDict([('G_GAN', 6.7868452072143555), ('G_L1', 26.793216705322266), ('D', 0.6676896214485168), ('F', 0.06749366223812103)])
OrderedDict([('G_GAN', 6.387866020202637), ('G_L1', 19.964893341064453), ('D', 0.724107563495636), ('F', 0.057628292590379715)])
OrderedDict([('G_GAN', 6.263550281524658), ('G_L1', 34.09487533569336), ('D', 0.6789134740829468), ('F', 0.08398637175559998)])
OrderedDict([('G_GAN', 6.741687774658203), ('G_L1', 18.26243019104004), ('D', 0.699059009552002), ('F', 0.06891174614429474)])
OrderedDict([('G_GAN', 7.234025001525879), ('G_L1', 39.49540328979492), ('D', 0.8434057831764221), ('F', 0.0718810185790062)])
OrderedDict([('G_GAN', 7.337300777435303), ('G_L1', 19.778736114501953), ('D', 0.693669319152832), ('F', 0.06718950718641281)])
OrderedDict([('G_GAN', 6.389819622039795), ('G_L1', 23.649181365966797), ('D', 1.4450132846832275), ('F', 0.0715000256896019)])
OrderedDict([('G_GAN', 6.051386833190918), ('G_L1', 18.988412857055664), ('D', 1.0752019882202148), ('F

OrderedDict([('G_GAN', 6.731377601623535), ('G_L1', 20.863576889038086), ('D', 0.3464386761188507), ('F', 0.059839699417352676)])
OrderedDict([('G_GAN', 6.741943359375), ('G_L1', 22.257671356201172), ('D', 0.31236904859542847), ('F', 0.05939488112926483)])
OrderedDict([('G_GAN', 6.385989665985107), ('G_L1', 18.284292221069336), ('D', 0.4868622422218323), ('F', 0.10482761263847351)])
OrderedDict([('G_GAN', 5.530402660369873), ('G_L1', 37.732337951660156), ('D', 0.8944211006164551), ('F', 0.0596599206328392)])
OrderedDict([('G_GAN', 6.178887367248535), ('G_L1', 14.976432800292969), ('D', 0.6496527194976807), ('F', 0.04037339240312576)])
OrderedDict([('G_GAN', 5.604404449462891), ('G_L1', 30.673599243164062), ('D', 0.8652746677398682), ('F', 0.0462554469704628)])
OrderedDict([('G_GAN', 5.613526344299316), ('G_L1', 27.22817611694336), ('D', 0.9248689413070679), ('F', 0.05745531618595123)])
OrderedDict([('G_GAN', 6.711464881896973), ('G_L1', 18.09062957763672), ('D', 0.37643682956695557), (

...

OrderedDict([('G_GAN', 5.908272743225098), ('G_L1', 2.5178658962249756), ('D', 0.5804567337036133), ('F', 0.0034069623798131943)])
OrderedDict([('G_GAN', 5.144754409790039), ('G_L1', 2.17704701423645), ('D', 1.1624138355255127), ('F', 0.007833991199731827)])
OrderedDict([('G_GAN', 5.651724338531494), ('G_L1', 5.392729759216309), ('D', 0.8845289349555969), ('F', 0.002538531320169568)])
End of epoch 67 / 120 	 Time Taken: 461 sec
learning rate = 0.0001030
OrderedDict([('G_GAN', 5.108078956604004), ('G_L1', 2.090961217880249), ('D', 1.016742467880249), ('F', 0.002131764078512788)])
OrderedDict([('G_GAN', 4.939151287078857), ('G_L1', 2.4840903282165527), ('D', 1.197898268699646), ('F', 0.007486595306545496)])
OrderedDict([('G_GAN', 6.058821678161621), ('G_L1', 2.641472339630127), ('D', 0.4742783010005951), ('F', 0.00315950158983469)])
OrderedDict([('G_GAN', 5.90474271774292), ('G_L1', 4.824966907501221), ('D', 0.6250487565994263), ('F', 0.002430269494652748)])
OrderedDict([('G_GAN', 6.7316

OrderedDict([('G_GAN', 5.275346755981445), ('G_L1', 2.593708038330078), ('D', 0.9839756488800049), ('F', 0.005032879300415516)])
OrderedDict([('G_GAN', 5.631688594818115), ('G_L1', 3.162311315536499), ('D', 0.6405148506164551), ('F', 0.0024375906214118004)])
OrderedDict([('G_GAN', 5.553404808044434), ('G_L1', 2.7840805053710938), ('D', 0.9935581684112549), ('F', 0.00596788851544261)])
OrderedDict([('G_GAN', 5.800994873046875), ('G_L1', 4.9856791496276855), ('D', 0.7099899053573608), ('F', 0.006509477272629738)])
OrderedDict([('G_GAN', 5.8287353515625), ('G_L1', 4.131556987762451), ('D', 0.8496651649475098), ('F', 0.002279863692820072)])
OrderedDict([('G_GAN', 5.211762428283691), ('G_L1', 2.0578010082244873), ('D', 0.996996283531189), ('F', 0.0023859101347625256)])
OrderedDict([('G_GAN', 6.694875717163086), ('G_L1', 4.009796142578125), ('D', 0.1695539355278015), ('F', 0.04642363265156746)])
OrderedDict([('G_GAN', 4.826973915100098), ('G_L1', 2.2868454456329346), ('D', 1.5165600776672363

OrderedDict([('G_GAN', 5.102321624755859), ('G_L1', 2.3964684009552), ('D', 1.082044243812561), ('F', 0.002598968567326665)])
OrderedDict([('G_GAN', 5.264014720916748), ('G_L1', 3.0620248317718506), ('D', 1.0794849395751953), ('F', 0.0025361631996929646)])
OrderedDict([('G_GAN', 5.789815902709961), ('G_L1', 2.382660388946533), ('D', 0.6043788194656372), ('F', 0.006590498145669699)])
OrderedDict([('G_GAN', 5.564040184020996), ('G_L1', 3.301133155822754), ('D', 0.7575976848602295), ('F', 0.0035483858082443476)])
OrderedDict([('G_GAN', 5.588264465332031), ('G_L1', 2.7962470054626465), ('D', 0.6944161653518677), ('F', 0.0016818749718368053)])
OrderedDict([('G_GAN', 5.169253826141357), ('G_L1', 2.4696338176727295), ('D', 1.636814832687378), ('F', 0.0030167875811457634)])
OrderedDict([('G_GAN', 5.529941082000732), ('G_L1', 2.41363525390625), ('D', 0.7828758358955383), ('F', 0.003164517693221569)])
OrderedDict([('G_GAN', 6.506650924682617), ('G_L1', 3.921928882598877), ('D', 0.328831315040588

OrderedDict([('G_GAN', 5.403641700744629), ('G_L1', 3.316056728363037), ('D', 0.8361403942108154), ('F', 0.01013239100575447)])
OrderedDict([('G_GAN', 5.404499530792236), ('G_L1', 2.7477810382843018), ('D', 0.9340969920158386), ('F', 0.0024336185306310654)])
OrderedDict([('G_GAN', 5.332881927490234), ('G_L1', 3.3104896545410156), ('D', 0.928956151008606), ('F', 0.001272865803912282)])
OrderedDict([('G_GAN', 5.675417423248291), ('G_L1', 3.299100399017334), ('D', 0.8065384030342102), ('F', 0.001509004971012473)])
OrderedDict([('G_GAN', 4.835282325744629), ('G_L1', 2.6706807613372803), ('D', 1.1892224550247192), ('F', 0.01159570924937725)])
OrderedDict([('G_GAN', 5.531599044799805), ('G_L1', 2.856153964996338), ('D', 0.7422715425491333), ('F', 0.0013179280795156956)])
OrderedDict([('G_GAN', 5.79292631149292), ('G_L1', 3.743316888809204), ('D', 0.616426944732666), ('F', 0.0022189426235854626)])
OrderedDict([('G_GAN', 5.564029693603516), ('G_L1', 3.3520209789276123), ('D', 0.721745848655700

OrderedDict([('G_GAN', 5.480616569519043), ('G_L1', 5.234231948852539), ('D', 0.9550658464431763), ('F', 0.0031595169566571712)])
OrderedDict([('G_GAN', 6.487822532653809), ('G_L1', 3.6683220863342285), ('D', 0.4407646059989929), ('F', 0.004324716050177813)])
OrderedDict([('G_GAN', 5.555498123168945), ('G_L1', 4.607581615447998), ('D', 0.9280225038528442), ('F', 0.0052385833114385605)])
OrderedDict([('G_GAN', 5.102975845336914), ('G_L1', 2.3822879791259766), ('D', 1.1212457418441772), ('F', 0.002245933748781681)])
OrderedDict([('G_GAN', 5.40559720993042), ('G_L1', 2.640793561935425), ('D', 1.0778014659881592), ('F', 0.002907639369368553)])
OrderedDict([('G_GAN', 5.390935897827148), ('G_L1', 2.9666051864624023), ('D', 1.0540771484375), ('F', 0.004658578895032406)])
OrderedDict([('G_GAN', 6.926242828369141), ('G_L1', 6.756328105926514), ('D', 0.3575069308280945), ('F', 0.06367383897304535)])
OrderedDict([('G_GAN', 5.189150810241699), ('G_L1', 1.989604115486145), ('D', 0.9835339784622192)